# 3rd Capstone Notebook
# More Spacy test processing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1000000
from collections import Counter
import gender_guesser.detector as gender
d = gender.Detector(case_sensitive=False)

In [182]:
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('~/OneDrive/Documents/BrainStation/Capstone/final_final.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,genres,ISBN,author,Year-Of-Publication,Publisher,Publication_year,author_first,author_last,author_gender
0,1,the house on the borderland,"Produced by Suzanne Shell, Sjaani and PG Distr...","{'horror', 'mystery', 'classics', 'science-fic...",0786702826,william hope hodgson,1996,Carroll &amp; Graf Publishers,1996,william,hope hodgson,male
1,2,the warriors,"Produced by Charles Aldarondo, Charlie Kirschn...","{'literary-fiction', 'history', 'biography', '...",0802139922,sol yurick,2003,Grove Press,2003,sol,yurick,female
2,4,carmilla,"Produced by Suzanne Shell, Sjaani and PG Distr...","{'romance', 'novella', 'historical-fiction', '...",1843500728,joseph sheridan le fanu,2003,Soft Editions Ltd Pod,2003,joseph,sheridan le fanu,male
3,5,discourses,"Produced by Ted Garvin, Jayam Subramanian and ...","{'education', 'history', 'roman', 'classics', ...",0140444289,niccolo machiavelli,1984,Penguin Books,1984,niccolo,machiavelli,unknown
4,6,the constitution,"Produced by Afra Ullah, Dave Morgan and PG Dis...","{'history', 'classics', 'reference', 'philosop...",0688083498,page smith,1980,Harpercollins,1980,page,smith,unknown


In [4]:
df.shape

(1090, 12)

In [5]:
df.drop(columns='Unnamed: 0', inplace=True)

In [6]:
df.head()

,title,text,genres,ISBN,author,Year-Of-Publication,Publisher,Publication_year,author_first,author_last,author_gender
0,the house on the borderland,"Produced by Suzanne Shell, Sjaani and PG Distr...","{'horror', 'mystery', 'classics', 'science-fic...",0786702826,william hope hodgson,1996,Carroll &amp; Graf Publishers,1996,william,hope hodgson,male
1,the warriors,"Produced by Charles Aldarondo, Charlie Kirschn...","{'literary-fiction', 'history', 'biography', '...",0802139922,sol yurick,2003,Grove Press,2003,sol,yurick,female
2,carmilla,"Produced by Suzanne Shell, Sjaani and PG Distr...","{'romance', 'novella', 'historical-fiction', '...",1843500728,joseph sheridan le fanu,2003,Soft Editions Ltd Pod,2003,joseph,sheridan le fanu,male
3,discourses,"Produced by Ted Garvin, Jayam Subramanian and ...","{'education', 'history', 'roman', 'classics', ...",0140444289,niccolo machiavelli,1984,Penguin Books,1984,niccolo,machiavelli,unknown
4,the constitution,"Produced by Afra Ullah, Dave Morgan and PG Dis...","{'history', 'classics', 'reference', 'philosop...",0688083498,page smith,1980,Harpercollins,1980,page,smith,unknown


In [7]:
for book in df['text']:
    print(len(book))

280367
276587
155664
799865
204495
510752
198895
629459
128080
662899
67799
462419
527158
282034
43317
412070
876799
128659
467008
161193
21666
20444
20154
86589
404308
146845
220526
343077
1030547
19644
537721
48255
806179
7340
638085
389093
437572
258672
244299
38310
828712
904923
305315
761821
224881
153828
63738
730002
179650
443543
190300
279441
414481
8404
375054
362928
106715
161308
44942
432654
136919
824565
221972
577973
633240
174467
23756
23883
202972
202325
557003
426223
700867
185835
273980
633371
505983
425287
529967
277581
2612113
598711
274244
284121
369979
338052
304123
185032
58523
150464
435897
172824
3813
23802
684080
18350
19371
21731
16456
20407
193805
571854
25554
843956
281533
1879721
361553
690618
229524
297162
946325
573686
195407
356649
97517
402903
223297
1295796
465566
213498
471620
411456
104186
243687
38544
490406
17157
132755
610549
252297
529639
43218
711743
287108
579188
338588
104233
542188
110582
685977
484044
350147
411014
143500
1057544
461510
5150

In [8]:
text_to_split = df['text']

In [238]:
splitted_texts = []
for t in text_to_split:
    splitted = t.split('\n')
    splitted_texts.append(splitted)

## Experiment with saving only what I need

### what I need
- named entities == 'PERSON'
- top named entities
- verbs associated with top named entities

In [186]:
carmilla = nlp(str(splitted_texts[2]))

In [292]:
def identify_main_characters(doc_name):
    # get all characters and how often they appear
    characters = []
    for ent in doc_name.ents:
        if ent.label_ == 'PERSON':
            characters.append(ent.text)
    top_chars = {}
    char_count = Counter(characters)
    
    # get main characters and how often they appear
    for chars, count in char_count.items():
        if count >= 5:
            top_chars[chars] = count
    
    # get verbs associated with main characters
    verbs = {}
    for token in doc_name:
        for chars in top_chars:
            if token.text == chars:
                if token.dep_ == 'nsubj':
                    if chars not in verbs:
                        verbs[chars] = []
                    verbs[chars].append(token.head.text)
    
    return char_count, top_chars, verbs

In [250]:
# version of loop with df output
first_split = splitted_texts[:2]
results = []
what_i_want = pd.DataFrame(pd.DataFrame(columns=['doc_name', 'characters', 'main_characters', 'character_verbs']))

for bits in first_split:
    processed_bits = nlp(str(bits))
    info_from_func = identify_main_characters(processed_bits)
    new_row = {'doc_name': bits, 'characters': char_count, 'main_characters': top_chars, 'character_verbs': verbs}
    what_i_want.loc[len(what_i_want)] = new_row
    print(f'characters {char_count}')
    
    print(f'main characters {top_chars}')
    
    print(f'character verbs {verbs}')

characters Counter({'Carmilla': 13, 'Karnstein': 11, 'Baron': 6, 'Spielsdorf': 5, 'Mircalla': 5, 'Countess Karnstein': 5, 'Karnsteins': 5, 'Laura': 5, 'Mademoiselle De Lafontaine': 4, 'Countess': 3, 'Vordenburg': 3, 'Hesselius': 2, 'Essay': 2, 'Bertha': 2, 'Martin': 2, 'Millarca': 2, "J. Sheridan LeFanu'": 2, 'Suzanne Shell': 1, 'Guest': 1, 'Mademoiselle Rheinfeldt': 1, 'Shakespeare': 1, 'Matska': 1, 'Cleopatra': 1, 'Saunter': 1, 'hymn sung': 1, 'buff': 1, 'wolf': 1, 'Monsieur Buffon': 1, 'Papa': 1, 'Marcia': 1, 'Descending': 1, 'Search': 1, 'Spielsberg': 1, 'shaggy grey': 1, 'Carlsfeld': 1, 'Beside': 1, "\\'Then": 1, 'The Woodman': 1, 'Gothic': 1, 'Mademoiselle Carmilla': 1, 'Ordeal': 1, 'Magia Posthuma': 1, 'John Christofer Herenberg': 1, "Torlogh O'Brien": 1, 'Churchyard': 1, 'Sheridan LeFanu': 1, 'Mysteries': 1})
main characters {'Spielsdorf': 5, 'Karnstein': 11, 'Carmilla': 13, 'Mircalla': 5, 'Countess Karnstein': 5, 'Karnsteins': 5, 'Laura': 5, 'Baron': 6}
character verbs {'Spiel

In [293]:
# version of loop with dicts output
first_split = splitted_texts[:2]
results = []

for bits in first_split:
    processed_bits = nlp(' '.join(bits))  # Join the tokens into a single string
    info_from_func = identify_main_characters(processed_bits)
    results.append((info_from_func))

In [294]:
results

[(Counter({'Suzanne Shell': 1,
           'Manuscript': 2,
           'Tonnison': 37,
           'Berreggnog': 1,
           'Hush': 5,
           'Dead': 1,
           'Kraighten': 1,
           'Ardrahan': 2,
           'gaunt': 2,
           'Hitherto': 3,
           'débris': 2,
           'II': 1,
           'Afar': 2,
           'Seth': 1,
           'IV': 1,
           'V': 1,
           'Pit': 40,
           'Jove': 1,
           'bush': 1,
           'Mary': 10,
           'VI': 1,
           'VII': 1,
           'cr-ac-k': 1,
           'Quickly': 2,
           'XI': 1,
           'XIV': 1,
           'XV': 1,
           'Westward': 7,
           'Northward': 1,
           'Faster': 1,
           'Outer': 1,
           'decay': 1,
           'XVII': 1,
           'Longer': 1,
           'XVIII': 1,
           'Green': 1,
           'Green Sun': 1,
           'Green Orb': 1,
           'XX': 1,
           'Thou': 1,
           'XXI': 1,
           'Specter': 1,
           'XXI

In [290]:
def identify_main_characters(doc_name):
    # get all characters and how often they appear
    characters = []
    for ent in doc_name.ents:
        if ent.label_ == 'PERSON':
            characters.append(ent.text)
    top_chars = {}
    char_count = Counter(characters)
    
    # get main characters and how often they appear
    for chars, count in char_count.items():
        if count >= 5:
            top_chars[chars] = count
    
    # get verbs associated with main characters
    verbs = {}
    for token in doc_name:
        for chars in top_chars:
            if token.text == chars:
                print("Matched character:", chars)
                if token.dep_ == 'nsubj':
                    print("Matched nsubj token:", token.text)
                    if chars not in verbs:
                        verbs[chars] = []
                    verbs[chars].append(token.head.text)
    
    return char_count, top_chars, verbs

In [291]:
identify_main_characters(house)

Matched character: Tonnison
Matched character: Hush
Matched character: Hush
Matched character: Hush
Matched character: Hush
Matched character: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched nsubj token: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched character: Tonnison
Matched characte

(Counter({'Suzanne Shell': 1,
          'William Hope Hodgson': 2,
          'Manuscript': 2,
          'Tonnison': 39,
          'Berreggnog': 1,
          'Kraighten': 2,
          'Hush': 5,
          'Dead': 1,
          'Ardrahan': 2,
          'gaunt': 2,
          'Hitherto': 3,
          'débris': 2,
          'II': 1,
          'Seth': 1,
          'IV': 1,
          'V': 1,
          'Pit': 40,
          'Jove': 1,
          'bush': 1,
          'Mary': 8,
          'VI': 1,
          'Babel': 1,
          'VII': 1,
          'IX': 1,
          'XI': 1,
          'XIV': 1,
          'XV': 1,
          'Westward': 8,
          'Northward': 1,
          'Faster': 1,
          'XVI': 1,
          'Outer': 1,
          'decay': 1,
          'XVII': 1,
          'XVIII': 1,
          'XIX': 1,
          'Green': 1,
          'Fascinated': 1,
          'Green Sun': 2,
          'Green Orb': 1,
          'XX': 1,
          'Thou': 1,
          'XXI': 1,
          'Specter': 1,
     

In [288]:
# version of loop with dicts output
# + troubleshoot
first_split = splitted_texts[:2]
results = []

for bits in first_split:
    processed_bits = nlp(' '.join(bits))  # Join the tokens into a single string
    info_from_func = identify_main_characters(processed_bits)
    results.append((info_from_func))

Matched character: Tonnison


KeyError: 'Tonnison'

In [280]:
results

[(Counter({'Suzanne Shell': 1,
           'William Hope Hodgson': 2,
           'Manuscript': 2,
           'Tonnison': 39,
           'Berreggnog': 1,
           'Kraighten': 2,
           'Hush': 5,
           'Dead': 1,
           'Ardrahan': 2,
           'gaunt': 2,
           'Hitherto': 3,
           'débris': 2,
           'II': 1,
           'Seth': 1,
           'IV': 1,
           'V': 1,
           'Pit': 40,
           'Jove': 1,
           'bush': 1,
           'Mary': 8,
           'VI': 1,
           'Babel': 1,
           'VII': 1,
           'IX': 1,
           'XI': 1,
           'XIV': 1,
           'XV': 1,
           'Westward': 8,
           'Northward': 1,
           'Faster': 1,
           'XVI': 1,
           'Outer': 1,
           'decay': 1,
           'XVII': 1,
           'XVIII': 1,
           'XIX': 1,
           'Green': 1,
           'Fascinated': 1,
           'Green Sun': 2,
           'Green Orb': 1,
           'XX': 1,
           'Thou': 1,
       

In [251]:
what_i_want

,doc_name,characters,main_characters,character_verbs
0,"[Produced by Suzanne Shell, Sjaani and PG Dist...","{'Suzanne Shell': 1, 'Hesselius': 2, 'Essay': ...","{'Spielsdorf': 5, 'Karnstein': 11, 'Carmilla':...","{'Spielsdorf': ['come', 'mentioned'], 'Carmill..."
1,"[Produced by Charles Aldarondo, Charlie Kirsch...","{'Suzanne Shell': 1, 'Hesselius': 2, 'Essay': ...","{'Spielsdorf': 5, 'Karnstein': 11, 'Carmilla':...","{'Spielsdorf': ['come', 'mentioned'], 'Carmill..."


In [243]:
results

[(Counter({'Suzanne Shell': 1,
           'William Hope Hodgson': 2,
           'Manuscript': 2,
           'Tonnison': 39,
           'Berreggnog': 1,
           'Kraighten': 2,
           'Hush': 5,
           'Dead': 1,
           'Ardrahan': 2,
           'gaunt': 2,
           'Hitherto': 3,
           'débris': 2,
           'II': 1,
           'Seth': 1,
           'IV': 1,
           'V': 1,
           'Pit': 40,
           'Jove': 1,
           'bush': 1,
           'Mary': 8,
           'VI': 1,
           'Babel': 1,
           'VII': 1,
           'IX': 1,
           'XI': 1,
           'XIV': 1,
           'XV': 1,
           'Westward': 8,
           'Northward': 1,
           'Faster': 1,
           'XVI': 1,
           'Outer': 1,
           'decay': 1,
           'XVII': 1,
           'XVIII': 1,
           'XIX': 1,
           'Green': 1,
           'Fascinated': 1,
           'Green Sun': 2,
           'Green Orb': 1,
           'XX': 1,
           'Thou': 1,
       

In [244]:
verbs

{'Spielsdorf': ['come', 'mentioned'],
 'Carmilla': ['said',
  'purchased',
  'said',
  'said',
  'sat',
  'is',
  'answered',
  'take',
  'playing',
  'came',
  'said',
  'became',
  'complained',
  'was',
  'was',
  'discovered',
  'was',
  'leaning',
  'looking',
  'restored',
  'hear',
  'seen',
  'approaching',
  'is',
  'entered',
  'made',
  'did'],
 'Laura': ['looking', 'come', 'come', 'be'],
 'Mircalla': ['seemed', 'closed']}

In [254]:
# Find the maximum length of the arrays
max_length = max(len(v) for v in verbs.values())

# Pad the shorter arrays with NaN values
verbs_padded = {k: v + [float('nan')] * (max_length - len(v)) for k, v in verbs.items()}

# Create a DataFrame from the padded dictionary
df_verbs = pd.DataFrame.from_dict(verbs_padded)
df_verbs

,Spielsdorf,Carmilla,Laura,Mircalla
0,come,said,looking,seemed
1,mentioned,purchased,come,closed
2,NaN,said,come,NaN
3,NaN,said,be,NaN
4,NaN,sat,NaN,NaN
5,NaN,is,NaN,NaN
6,NaN,answered,NaN,NaN
7,NaN,take,NaN,NaN
8,NaN,playing,NaN,NaN
9,NaN,came,NaN,NaN


In [230]:
identify_characters(carmilla)
print(char_count)
print(top_chars)
print(verbs)

Counter({'Carmilla': 13, 'Karnstein': 11, 'Baron': 6, 'Spielsdorf': 5, 'Mircalla': 5, 'Countess Karnstein': 5, 'Karnsteins': 5, 'Laura': 5, 'Mademoiselle De Lafontaine': 4, 'Countess': 3, 'Vordenburg': 3, 'Hesselius': 2, 'Essay': 2, 'Bertha': 2, 'Martin': 2, 'Millarca': 2, "J. Sheridan LeFanu'": 2, 'Suzanne Shell': 1, 'Guest': 1, 'Mademoiselle Rheinfeldt': 1, 'Shakespeare': 1, 'Matska': 1, 'Cleopatra': 1, 'Saunter': 1, 'hymn sung': 1, 'buff': 1, 'wolf': 1, 'Monsieur Buffon': 1, 'Papa': 1, 'Marcia': 1, 'Descending': 1, 'Search': 1, 'Spielsberg': 1, 'shaggy grey': 1, 'Carlsfeld': 1, 'Beside': 1, "\\'Then": 1, 'The Woodman': 1, 'Gothic': 1, 'Mademoiselle Carmilla': 1, 'Ordeal': 1, 'Magia Posthuma': 1, 'John Christofer Herenberg': 1, "Torlogh O'Brien": 1, 'Churchyard': 1, 'Sheridan LeFanu': 1, 'Mysteries': 1})
{'Spielsdorf': 5, 'Karnstein': 11, 'Carmilla': 13, 'Mircalla': 5, 'Countess Karnstein': 5, 'Karnsteins': 5, 'Laura': 5, 'Baron': 6}
{'Spielsdorf': ['come', 'mentioned'], 'Carmilla': 

In [187]:
# collect characters
characters = []
for ent in carmilla.ents:
    if ent.label_ == 'PERSON':
        characters.append(ent.text)
print(characters)

['Suzanne Shell', 'Hesselius', 'Essay', 'Essay', 'Hesselius', 'Spielsdorf', 'Karnstein', 'Mademoiselle De Lafontaine', 'Guest', 'Spielsdorf', 'Mademoiselle Rheinfeldt', 'Spielsdorf', 'Spielsdorf', 'Bertha', 'Bertha', 'Mademoiselle De Lafontaine', 'Shakespeare', 'Matska', 'Mademoiselle De Lafontaine', 'Cleopatra', 'Saunter', 'Carmilla', 'hymn sung', 'buff', 'wolf', 'Carmilla', 'Carmilla', 'Monsieur Buffon', 'Karnstein', 'Papa', 'Marcia', 'Mircalla', 'Countess Karnstein', 'Karnsteins', 'Karnsteins', 'Carmilla', 'Carmilla', 'Carmilla', 'Mademoiselle De Lafontaine', 'Carmilla', 'Descending', 'Martin', 'Martin', 'Carmilla', 'Carmilla', 'Search', 'Carmilla', 'Carmilla', 'Laura', 'Carmilla', 'Laura', 'Laura', 'Spielsberg', 'Laura', 'Laura', 'Spielsdorf', 'Karnstein', 'Karnstein', 'shaggy grey', 'Karnstein', 'Karnstein', 'Karnsteins', 'Karnstein', 'Carlsfeld', 'Karnstein', 'Beside', "\\'Then", 'Countess', 'Baron', 'Millarca', 'The Woodman', 'Karnsteins', 'Karnsteins', 'Karnstein', 'Millarca', 

In [199]:
# put top characters in dictionary
top_chars = {}
char_count = Counter(characters)
for chars, count in char_count.items():
    if count >= 5:
        top_chars[chars] = count

In [200]:
top_chars

{'Spielsdorf': 5,
 'Karnstein': 11,
 'Carmilla': 13,
 'Mircalla': 5,
 'Countess Karnstein': 5,
 'Karnsteins': 5,
 'Laura': 5,
 'Baron': 6}

In [214]:
# take verbs associated with top_chars
verbs = {}
for token in carmilla:
    for chars in top_chars:
        if token.text == chars:
            if token.dep_ == 'nsubj':
                if chars not in verbs:
                    verbs[chars] = []
                verbs[chars].append(token.head.text)
                print(token.text, token.dep_, token.head.text)

Spielsdorf nsubj come
Spielsdorf nsubj mentioned
Carmilla nsubj said
Carmilla nsubj purchased
Carmilla nsubj said
Carmilla nsubj said
Carmilla nsubj sat
Carmilla nsubj is
Carmilla nsubj answered
Carmilla nsubj take
Carmilla nsubj playing
Carmilla nsubj came
Carmilla nsubj said
Carmilla nsubj became
Carmilla nsubj complained
Carmilla nsubj was
Carmilla nsubj was
Carmilla nsubj discovered
Carmilla nsubj was
Carmilla nsubj leaning
Carmilla nsubj looking
Laura nsubj looking
Carmilla nsubj restored
Carmilla nsubj hear
Laura nsubj come
Laura nsubj come
Laura nsubj be
Carmilla nsubj seen
Carmilla nsubj approaching
Carmilla nsubj is
Carmilla nsubj entered
Carmilla nsubj made
Mircalla nsubj seemed
Carmilla nsubj did
Mircalla nsubj closed


In [259]:
house_borderland = splitted_texts[0]
house = nlp(str(house_borderland))

In [266]:
# collect characters
characters_h = []
for ent in house.ents:
    if ent.label_ == 'PERSON':
        characters_h.append(ent.text)
print(characters_h)

['Suzanne Shell', 'William Hope Hodgson', 'Manuscript', 'Tonnison', 'Berreggnog', 'Kraighten', 'Hush', 'Hush', 'Hush', 'Hush', 'Dead', 'Kraighten', 'Tonnison', 'Ardrahan', 'Ardrahan', 'Tonnison', 'Tonnison', 'gaunt', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Hitherto', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'débris', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'Tonnison', 'II', 'Hitherto', 'Seth', 'IV', 'V', 'Pit', 'Pit', 'Pit', 'Pit', 'Jove', 'bush', 'Mary', 'VI', 'Pit', 'Pit', 'Pit', 'Mary', 'Mary', 'Pit', 'Pit', 'Pit', 'Babel', 'VII', 'IX', 'Mary', 'Mary', 'Mary', 'XI', 'Hitherto', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'Pit', 'XIV', 'Mary', 'gaunt', 'XV', 'Westward', 'Westward', 'Westwar

In [267]:
# put top characters in dictionary
top_chars_h = {}
char_count_h = Counter(characters_h)
for chars, count in char_count_h.items():
    if count >= 5:
        top_chars_h[chars] = count

In [268]:
top_chars_h

{'Tonnison': 39, 'Hush': 5, 'Pit': 40, 'Mary': 8, 'Westward': 8}

In [276]:
# take verbs associated with top_chars experiement for 1st row
verbs_h = {}
for token in house:
    for chars in top_chars_h:
        if token.text == chars:
            if token.dep_ == 'nsubj':
                if chars not in verbs_h:
                    verbs_h[chars] = []
                verbs_h[chars].append(token.head.text)
                print(token.text, token.dep_, token.head.text)

Tonnison nsubj got
Tonnison nsubj calling
Tonnison nsubj grunted
Tonnison nsubj said
Tonnison nsubj stopped
Tonnison nsubj answered
Tonnison nsubj was
Tonnison nsubj bent
Tonnison nsubj called
Tonnison nsubj said
Tonnison nsubj put
Tonnison nsubj spoke
Tonnison nsubj said
Tonnison nsubj began
Tonnison nsubj asked
Mary nsubj appeared
Pit nsubj came
Tonnison nsubj stared
Tonnison nsubj rose
Tonnison nsubj was
Tonnison nsubj was
Tonnison nsubj asked
Tonnison nsubj looked
Tonnison nsubj said
Tonnison nsubj sent


In [277]:
verbs_h

{'Tonnison': ['got',
  'calling',
  'grunted',
  'said',
  'stopped',
  'answered',
  'was',
  'bent',
  'called',
  'said',
  'put',
  'spoke',
  'said',
  'began',
  'asked',
  'stared',
  'rose',
  'was',
  'was',
  'asked',
  'looked',
  'said',
  'sent'],
 'Mary': ['appeared'],
 'Pit': ['came']}

In [215]:
verbs

{'Spielsdorf': ['come', 'mentioned'],
 'Carmilla': ['said',
  'purchased',
  'said',
  'said',
  'sat',
  'is',
  'answered',
  'take',
  'playing',
  'came',
  'said',
  'became',
  'complained',
  'was',
  'was',
  'discovered',
  'was',
  'leaning',
  'looking',
  'restored',
  'hear',
  'seen',
  'approaching',
  'is',
  'entered',
  'made',
  'did'],
 'Laura': ['looking', 'come', 'come', 'be'],
 'Mircalla': ['seemed', 'closed']}

## New Perform NLP on all books

In [293]:
# version of loop with dicts output
final_split = splitted_texts[:]
results = []

for bits in first_split:
    processed_bits = nlp(' '.join(bits))  # Join the tokens into a single string
    info_from_func = identify_main_characters(processed_bits)
    results.append((info_from_func))

## Perform NLP on all books

In [152]:
splitted_texts[2]

['Produced by Suzanne Shell, Sjaani and PG Distributed Proofreaders',
 '',
 '',
 '',
 '',
 'CARMILLA',
 '',
 'J. Sheridan LeFanu',
 '',
 '1872',
 '',
 '',
 '',
 'PROLOGUE',
 '',
 '_Upon a paper attached to the Narrative which follows, Doctor Hesselius',
 'has written a rather elaborate note, which he accompanies with a',
 'reference to his Essay on the strange subject which the MS. illuminates.',
 '',
 'This mysterious subject he treats, in that Essay, with his usual',
 'learning and acumen, and with remarkable directness and condensation. It',
 "will form but one volume of the series of that extraordinary man's",
 'collected papers.',
 '',
 'As I publish the case, in this volume, simply to interest the "laity," I',
 'shall forestall the intelligent lady, who relates it, in nothing; and',
 'after due consideration, I have determined, therefore, to abstain from',
 "presenting any précis of the learned Doctor's reasoning, or extract from",
 'his statement on a subject which he describes 

In [9]:
splitted_texts2 = []
for t in text_to_split:
    splitted2 = t.split('\n\n')
    splitted_texts2.append(splitted2)

In [12]:
splitted_texts2[20]

['Produced by David Widger',
 '',
 "\nSHIP'S COMPANY",
 'By W.W. Jacobs',
 '',
 'GOOD INTENTIONS',
 '\n"Jealousy; that\'s wot it is," said the night-watchman, trying to sneer--\n"pure jealousy."  He had left his broom for a hurried half-pint at the\n"Bull\'s Head"--left it leaning in a negligent attitude against the\nwarehouse-wall; now, lashed to the top of the crane at the jetty end, it\npointed its soiled bristles towards the evening sky and defied capture.',
 '"And I know who it is, and why \'e\'s done it," he continued.  "Fust and\nlast, I don\'t suppose I was talking to the gal for more than ten minutes,\nand \'arf of that was about the weather.',
 '"I don\'t suppose anybody \'as suffered more from jealousy than wot I \'ave:\nOther people\'s jealousy, I mean.  Ever since I was married the missis has\nbeen setting traps for me, and asking people to keep an eye on me.  I\nblacked one of the eyes once--like a fool--and the chap it belonged to\nmade up a tale about me that I ain\'t l

In [146]:
for pieces in splitted_texts2:
    processed = nlp.pipe(pieces)

In [147]:
pieces

['Produced by Ralph Zimmermann, Charles Franks\nand the Online Distributed Proofreading Team.',
 '',
 '',
 '\n[Illustration with caption: OVERPOWERING CURIOSITY OF A MOUNTAIN SHEEP\nThis "lava ram" stood thus on a lava crest in the Pinacate Mountains\nfor about twenty minutes, gazing spellbound at two men and a pack mule.\n(See page 149)]',
 '',
 '\nTHE MINDS AND MANNERS OF WILD ANIMALS',
 'A BOOK OF PERSONAL OBSERVATIONS',
 'BY WILLIAM T. HORNADAY, Sc.D., A.M. DIRECTOR OF THE NEW YORK\nZOOLOGICAL PARK. AUTHOR OF "THE AMERICAN NATURAL HISTORY," "TWO\nYEARS IN THE JUNGLE," "CAMP FIRES IN THE CANADIAN ROCKIES," "OUR\nVANISHING WILD LIFE," ETC.',
 'WITH ILLUSTRATIONS',
 '_The wild animal must think, or die._* * * * *',
 '_"Prove all things; hold fast that which is good."_',
 'COPYRIGHT, 1922, BY WILLIAM T. HORNADAY',
 'Printed in the United States of America',
 '_The right of translation is reserved_',
 'Published May, 1922',
 '',
 '\nTO THE OFFICERS AND MEN OF THE NEW YORK ZOOLOGICAL PAR

In [148]:
processed

<generator object Language.pipe at 0x0000024CDBDAEB90>

In [142]:
next(processed)


[Illustration with caption: OVERPOWERING CURIOSITY OF A MOUNTAIN SHEEP
This "lava ram" stood thus on a lava crest in the Pinacate Mountains
for about twenty minutes, gazing spellbound at two men and a pack mule.
(See page 149)]

In [143]:
for gen in processed:
    for token in gen:
        if token.ent_type_=='PERSON':
            characters = token.text, token.ent_type_

In [144]:
len(characters)

2

In [145]:
characters

('Hornaday', 'PERSON')

In [181]:
# Cancelled, took 3+ hours
docs = []
for books in splitted_texts2:
    docs = list(nlp.pipe(books, n_process=4, batch_size=100))

KeyboardInterrupt: 

In [ ]:
docs

In [ ]:
novels

In [ ]:
len(docs)

In [177]:
for books in splitted_texts:
    novels = nlp.pipe(books)

In [178]:
novels

<generator object Language.pipe at 0x0000024CDBDAE500>

In [179]:
next(novels)

Produced by Ralph Zimmermann, Charles Franks

In [180]:
for gen in novels:
    for token in gen:
        if token.ent_type_ == 'PERSON':
            if token.dep_ == "nsubj":
                print(token.text, token.dep_, token.head.text)

Spofford nsubj used
Romanes nsubj published
Yerkes nsubj develop
Gorilla nsubj was
deer nsubj is
Gibson nsubj wrote
Eskimo nsubj excited
Hippopotamus nsubj is
Wah nsubj comes
Alice nsubj marched
Horse nsubj is
brook nsubj ran
Hans nsubj knew
Hans nsubj was
Hans nsubj came
Hans nsubj give
Hans nsubj remember
Hans nsubj was
Stefansson nsubj says
Smith nsubj that
wolf nsubj are
Wright nsubj says
Billy nsubj has
Eskimo nsubj is
Borneo nsubj build
Tennan nsubj entitled
Theobald,--and nsubj presented
Rajah nsubj was
Rajah nsubj pause
Rajah nsubj spoil
Balentidium nsubj coli
Dohong nsubj tested
Dohong nsubj pulled
Dohong nsubj started
Dohong nsubj turned
Dohong nsubj been
Joe nsubj was
Peter nsubj entered
Peter nsubj stopped
Peter nsubj perform
Peter nsubj went
Witmer nsubj uses
Peter nsubj got
Witmer nsubj taught
Witmer nsubj declares
Suzette nsubj astonished
Suzette nsubj walked
Suzette nsubj change
Suzette nsubj felt
Garden nsubj was
John nsubj was
Cunningham nsubj wrote
Cunningham nsubj s

In [ ]:
gen = docs[10]  # Get the specific generator from the list

for doc in gen:
    for token in doc:
        print(token.text, token.dep_, token.head.text)

In [ ]:
gen = docs[10]  # Get the specific generator from the list

for doc in gen:
    for token in doc:
        print(token.text, token.dep_, token.head, dir(token))

In [ ]:
docs[10]

In [ ]:
for gen in docs[10]:
    print(gen.text)

In [ ]:
gen = next(docs)  # Get the next generator from the list

for doc in gen:
    for token in doc:
        if token.ent_type_ == 'PERSON':
            if token.dep_ == "nsubj":
                print(token.text, token.dep_, token.head.text)

In [ ]:
for gen in docs:
    for doc in gen:
        for entity in doc.ents:
            if entity.label_ == 'PERSON':
                if token.dep_ == 'nsubj':
                    print(token.text, token.dep_, token.head.text)
        #if token.ent.label_ == 'PERSON':
         #   if token.dep_ == "nsubj":
          #      print(token.text, token.dep_, token.head.text)

In [ ]:
for gen in docs:
    for doc in gen:
        for entity in doc.ents:
            print(entity.text, entity.label_)


In [ ]:
# tried and failed. NLP without pipe
docs = []
for books in df['text']:
    novel = nlp(str(books))
    docs.append(novel)

In [ ]:
doc1 = next(docs)

In [ ]:
doc1 = nlp(str(df['text'][50]))

In [ ]:
for token in docs[51]:
    if token.ent_type_ == 'PERSON':
        if token.dep_ == 'nsubj':
            print(token.text, token.dep_, token.head.text)